In [ ]:
cd "/content/drive/MyDrive/Midsem_task1"

/content/drive/.shortcut-targets-by-id/130/Midsem_task1


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
train_data = pd.read_csv('Constraint_English_Train _Comma.csv', index_col=0, header=0)
val_data = pd.read_csv('Constraint_English_Val_Comma.csv', index_col=0, header=0)

train_data.replace(to_replace=['fake', 'real'], value=[1, 0], inplace=True)
train_data['label'].value_counts()

val_data.replace(to_replace=['fake','real'], value=[1,0],inplace=True)
val_data['label'].value_counts()

x_train = train_data['tweet']
y_train = train_data['label']

x_val = val_data['tweet']
y_val = val_data['label']

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/Midsem_task1/Constraint_English_Test - Sheet1.csv')

In [ ]:
test_data

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...
...,...,...
2135,2136,#CoronaVirusUpdates: State-wise details of Tot...
2136,2137,Tonight 12(midnight) onwards Disaster Manageme...
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 E...
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


LSTM


In [ ]:
import tensorflow as tf
import keras
import keras.backend as K

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Concatenate, Conv2D, Flatten, Dense, Embedding, LSTM
from keras.models import Model

from sklearn.feature_extraction.text import CountVectorizer

from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Bidirectional
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D

In [ ]:
# def get_model():
input_1 = Input(shape=(x_train_seq.shape[1],))

common_embed = Embedding(name="synopsis_embedd",input_dim =len(t.word_index)+1, 
                      output_dim=300,weights=[embedding_matrix], 
                      input_length=x_train_seq.shape[1],trainable=False) 


lstm_1 = common_embed(input_1)

common_lstm = LSTM(64,return_sequences=True, activation="relu")

bidir = Bidirectional(common_lstm)
vector_1 = bidir(lstm_1)
vector_1 = Flatten()(vector_1)


x1 = Dense(2000, activation="relu")(vector_1)
out = Dense(1, activation="sigmoid", name = 'out')(x1)
model = Model([input_1], [out])

# inter_output_model = keras.Model(model.input, model.get_layer(name='conc_layer').output) 
                                  
model.compile(loss=["binary_crossentropy"], metrics=['acc'], optimizer=Adam(0.00001))
# return model

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1492)]            0         
_________________________________________________________________
synopsis_embedd (Embedding)  (None, 1492, 300)         5960400   
_________________________________________________________________
bidirectional (Bidirectional (None, 1492, 128)         186880    
_________________________________________________________________
flatten (Flatten)            (None, 190976)            0         
_________________________________________________________________
dense (Dense)                (None, 2000)              381954000 
_________________________________________________________________
out (Dense)                  (None, 1)                 2001      
Total params: 388,103,281
Trainable params: 382,142,881
Non-trainable params: 5,960,400
________________________________

In [ ]:
def get_emb_matrix(t):
  import numpy as np
  embeddings_index = {}
  f = open('/content/drive/MyDrive/MUStARD/data/glove.840B.300d.txt')
  for line in f:
      values = line.split()
      word = values[0]
      if(word in t.word_index):
        st=1
        if(len(values)>301):
          st += len(values) - 301
          coefs = np.asarray(values[st:], dtype='float32')
          embeddings_index[word] = coefs
        else:
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))

  not_present_list = []
  vocab_size = len(t.word_index) + 1
  print('Loaded %s word vectors.' % len(embeddings_index))
  embedding_matrix = np.zeros((vocab_size, len(embeddings_index['no'])))
  for word, i in t.word_index.items():
      if word in embeddings_index.keys():
          embedding_vector = embeddings_index.get(word)
      else:
          not_present_list.append(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
      else:
          embedding_matrix[i] = np.zeros(300)

  return embedding_matrix

Training Phase


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import numpy

t = Tokenizer()
t.fit_on_texts(x_train)
x_train_seq = t.texts_to_sequences(x_train)
x_val_seq = t.texts_to_sequences(x_val)
print("Vocab Size:", len(t.word_index))

arr=[]
for vec in x_train_seq:
  arr.append(len(vec))
max_len = max(arr)

x_train_seq = pad_sequences(x_train_seq, maxlen=max_len, padding='post')
x_val_seq = pad_sequences(x_val_seq,maxlen=max_len,padding='post')

embedding_matrix = get_emb_matrix(t)

Vocab Size: 19867
Found 11898 word vectors.
Loaded 11898 word vectors.


In [ ]:
with tf.device('/device:GPU:0'):
  model.fit(x_train_seq,y_train.values.reshape(-1,1),epochs=10,batch_size=64)

Epoch 1/10
101/101 [==============================] - 379s 4s/step - loss: 0.5423 - acc: 0.7413
Epoch 2/10
101/101 [==============================] - 376s 4s/step - loss: 0.4251 - acc: 0.8106
Epoch 3/10
101/101 [==============================] - 381s 4s/step - loss: 0.3578 - acc: 0.8572
Epoch 4/10
101/101 [==============================] - 363s 4s/step - loss: 0.3147 - acc: 0.8702
Epoch 5/10
101/101 [==============================] - 356s 4s/step - loss: 0.2886 - acc: 0.8818
Epoch 6/10
101/101 [==============================] - 356s 4s/step - loss: 0.2705 - acc: 0.8874
Epoch 7/10
101/101 [==============================] - 387s 4s/step - loss: 0.2536 - acc: 0.8941
Epoch 8/10
101/101 [==============================] - 379s 4s/step - loss: 0.2403 - acc: 0.8992
Epoch 9/10
101/101 [==============================] - 357s 4s/step - loss: 0.2295 - acc: 0.9083
Epoch 10/10
101/101 [==============================] - 364s 4s/step - loss: 0.2198 - acc: 0.9117


In [ ]:
results = model.evaluate(x_val_seq,y_val.values.reshape(-1,1))

67/67 [==============================] - 24s 355ms/step - loss: 0.4478 - acc: 0.7967


In [ ]:
predict = (model.predict(x_val_seq) > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val.values.reshape(-1,1),predict))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1120
           1       0.88      0.89      0.89      1020

    accuracy                           0.89      2140
   macro avg       0.89      0.89      0.89      2140
weighted avg       0.89      0.89      0.89      2140



In [ ]:
model.save('/content/drive/MyDrive/Midsem_task1')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Midsem_task1/assets


In [ ]:
import re
import emoji
# Remove hashtag while keeping hashtag text

def clean_text(text):
  text = re.sub(r'#','', text)
  # Remove HTML special entities (e.g. &amp;)
  text = re.sub(r'\&\w*;', '', text)
  # Remove tickers
  text = re.sub(r'\$\w*', '', text)
  # Remove hyperlinks
  text = re.sub(r'https?:\/\/.*\/\w*', '', text)
  # Remove whitespace (including new line characters)
  text = re.sub(r'\s\s+','', text)
  text = re.sub(r'[ ]{2, }',' ',text)
  # Remove URL, RT, mention(@)
  text=  re.sub(r'http(\S)+', '',text)
  text=  re.sub(r'http ...', '',text)
  text=  re.sub(r'(RT|rt)[ ]*@[ ]*[\S]+','',text)
  text=  re.sub(r'RT[ ]?@','',text)
  text = re.sub(r'@[\S]+','',text)
  text = re.sub(r'[^\w\s]', '', text) 

  text = re.sub('\\n', ' ',text)
  text = re.sub('\\r', ' ',text)
  text = re.sub('\\t', ' ',text)
  text = re.sub('<[^>]*>', ' ',text)

  # Remove emoji's
  text = emoji.demojize(text)
  text = text.replace(":"," ")
  text = ' '.join(text.split()) 
  text = re.sub("([^\x00-\x7F])+"," ",text)
  
  return text

In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 4.8MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=e4c2868793e66cdc5c4a4dbf91c3f8a4626def46488cdfa1ef17444e031ce91d
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

x_train = x_train.apply(clean_text)
x_train = x_train.apply(lambda row: nltk.word_tokenize(row))
x_train = x_train.apply(lemmatize_text)

x_val = x_val.apply(clean_text)
x_val = x_val.apply(lambda row: nltk.word_tokenize(row))
x_val =x_val.apply(lemmatize_text)

In [ ]:
x_test = test_data['tweet']

In [ ]:
x_test = x_test.apply(clean_text)
x_test = x_test.apply(lambda row: nltk.word_tokenize(row))
x_test = x_test.apply(lemmatize_text)

In [ ]:
def join_text(text):
  return ' '.join(text)

train_df_clean['text'] = train_df_clean['text'].apply(join_text)
eval_df_clean['text'] = eval_df_clean['text'].apply(join_text)

In [ ]:
test_df_clean['text'] = test_df_clean['text'].apply(join_text)

In [ ]:
train_df_clean = pd.concat([x_train,y_train],axis=1)

In [ ]:
eval_df_clean = pd.concat([x_val,y_val],axis=1)

In [ ]:
test_df_clean = pd.concat([x_test],axis=1)

In [ ]:
train_df_clean.columns = ['text','target']
eval_df_clean.columns = ['text','target']

In [ ]:
train_df_clean = pd.concat([train_df_clean,eval_df_clean])

In [ ]:
train_df_clean.groupby('target').size()

target
0    4480
1    4080
dtype: int64

In [ ]:
test_df_clean.columns = ['text']

In [ ]:
train_args = {
    # 'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs': 10,
    # 'evaluate_during_training_steps': 100,
    # 'save_eval_checkpoints': False,
    'train_batch_size': 32,
    # 'eval_batch_size': 64,
    'overwrite_output_dir': True,
    'fp16': False,
    'wandb_project': "visualization-demo"
}

'roberta', 'roberta-base'
# For Model in directory outputs-3           // the best model till now

('roberta', 'roberta-base')

In [ ]:
train_args = {
    'sliding_window': True,
    'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs': 10,
    'evaluate_during_training_steps': 100,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 64,
    'overwrite_output_dir': True,
    'fp16': False,
    'wandb_project': "visualization-demo",
    'stride': 0.8,
    'max_seq_length': 128
}


# For Model in directory outputs-3           // the best model till now

In [ ]:
train_df_clean.groupby('target').size()

target
0    3360
1    3060
dtype: int64

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 204kB 11.3MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 7.4MB 13.4MB/s 
     |████████████████████████████████| 1.1MB 36.9MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 1.4MB 42.4MB/s 
     |████████████████████████████████| 1.8MB 32.3MB/s 
     |████████████████████████████████| 2.9MB 38.6MB/s 
     |████████████████████████████████| 317kB 37.5MB/s 
     |████████████████████████████████| 163kB 33.9MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
     |████████████████████████████████| 112kB 42.8MB/s 
     |████████████████████████████████| 4.5MB 33.7MB/s 
     |████████████████████████████████| 890kB 32.9MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 133kB 30.3MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 122kB 41.2MB/s 


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 16.5MB/s 


In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
model_BERT = ClassificationModel('bertweet', 'vinai/bertweet-base', num_labels=2, use_cuda=True,cuda_device=0,args=train_args)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classi

In [ ]:
model_roberta = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
with tf.device('/device:GPU:0'):
  model_BERT.train_model(train_df_clean)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Exception: ignored

In [ ]:
import sklearn
predictions = model_BERT.predict(list(eval_df_clean['text']))
print(sklearn.metrics.classification_report(list(eval_df_clean['target']),predictions[0]))


              precision    recall  f1-score   support

           0       0.94      0.97      0.96      1120
           1       0.96      0.94      0.95      1020

    accuracy                           0.95      2140
   macro avg       0.95      0.95      0.95      2140
weighted avg       0.95      0.95      0.95      2140



Outputs - 2

In [ ]:
predictions = model_roberta.predict(list(eval_df_clean['text']))

In [ ]:
print(sklearn.metrics.classification_report(list(eval_df_clean['target']),predictions[0]))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1120
           1       0.95      0.94      0.95      1020

    accuracy                           0.95      2140
   macro avg       0.95      0.95      0.95      2140
weighted avg       0.95      0.95      0.95      2140



In [ ]:
import sklearn

In [ ]:
model = ClassificationModel('roberta', 'outputs-4/', args=train_args)

Outputs-1

In [ ]:
predictions = model.predict(list(eval_df_clean['text']))

In [ ]:
print(sklearn.metrics.classification_report(list(eval_df_clean['target']),predictions[0]))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1120
           1       0.95      0.94      0.94      1020

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140



In [ ]:
import csv
# id = test_data.iloc[:,0]
# predictions = model.predict((list(test_df_clean['text'])))[0]
with open('result_bert_clean_train.csv','r') as file:
  reader = csv.reader(file)
#  writer.writerow(["id","label"])
  for i in rows:
    if(la[i]==1):
      writer.writerow([id[i],'fake'])
    else:
       writer.writerow([id[i],'real'])

In [ ]:
result = pd.read_csv('/content/drive/MyDrive/Midsem_task1/result_bert_clean_trainVal.csv')
result

,id,label
0,1,real
1,2,fake
2,3,fake
3,4,real
4,5,real
...,...,...
2135,2136,real
2136,2137,real
2137,2138,real
2138,2139,real


In [ ]:
def convert(i):
    if(i==0):
        return 'real'
    return 'fake'

df = pd.read_csv(open('/content/drive/MyDrive/Midsem_task1/result_bert_clean_trainVal.csv', 'r'), header = 0)
df['label'] = df['label'].apply(convert)
df.to_csv('/content/drive/MyDrive/Midsem_task1/result_bert_clean_trainVal.csv', index=False)

In [ ]:
result.replace(to_replace=[1,0], value=['fake','real'],inplace=True)
result

,id,label
0,fake,real
1,2,fake
2,3,fake
3,4,real
4,5,real
...,...,...
2135,2136,real
2136,2137,real
2137,2138,real
2138,2139,real


In [ ]:
result.to_csv('/content/drive/MyDrive/Midsem_task1/result_bert_clean_trainVal.csv')

In [ ]:
len(predictions)

2140